# Exploracion de datos sobre cobertura movil en Colombia

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

In [2]:
df_cobertura_movil = pd.read_csv('data/Cobertura_móvil_por_tecnología,_departamento_y_municipio_por_proveedor_20251031.csv')
df_cobertura_movil.sample(5)


,AÑO,TRIMESTRE,PROVEEDOR,COD DEPARTAMENTO,DEPARTAMENTO,COD MUNICIPIO,MUNICIPIO,CABECERA MUNICIPAL,COD CENTRO POBLADO,CENTRO POBLADO,COBERTURA 2G,COBERTURA 3G,"COBERTURA HSPA+, HSPA+DC",COBERTUTA 4G,COBERTURA LTE,COBERTURA 5G
336937,2018,2,COMUNICACION CELULAR S A COMCEL S A,76,VALLE DEL CAUCA,76113.0,BUGALAGRANDE,N,76113008.0,GALICIA,N,N,N,N,N,N
306720,2020,4,EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP,63,QUINDÍO,63190.0,CIRCASIA,S,63190001.0,CIRCASIA,N,N,N,N,S,N
94647,2022,4,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,73,TOLIMA,73236.0,DOLORES,N,73236011.0,SAN PEDRO,S,S,N,N,N,N
374917,2017,3,COMUNICACION CELULAR S A COMCEL S A,47,MAGDALENA,47189.0,CIÉNAGA,S,47189001.0,CIÉNAGA,S,S,S,N,N,N
154439,2019,4,COLOMBIA MOVIL S.A ESP,47,MAGDALENA,47541.0,PEDRAZA,S,47541001.0,PEDRAZA,S,S,S,N,N,N


In [3]:
df_datos_geograficos = pd.read_csv(
    'data/DIVIPOLA_CentrosPoblados.csv', 
    encoding='latin-1', 
    sep=';'
)
df_datos_geograficos.sample(5)


,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
3157,23,CÓRDOBA,23580,PUERTO LIBERTADOR,23580009,SANTA FÉ DE LAS CLARAS,CP,"-75,687002","7,735934"
5046,50,META,50001,VILLAVICENCIO,50001002,RINCÓN DE POMPEYA,CP,"-73,365557","4,040189"
5825,54,NORTE DE SANTANDER,54001,SAN JOSÉ DE CÚCUTA,54001015,RICAURTE,CP,"-72,354328","8,026317"
3361,23,CÓRDOBA,23815,TUCHÍN,23815004,FLECHAS,CP,"-75,524587","9,258519"
6859,70,SUCRE,70678,SAN BENITO ABAD,70678033,LAS CHISPAS,CP,"-74,948691","8,559258"


In [4]:
cobertura_clean = df_cobertura_movil.drop_duplicates().copy()

cobertura_clean['COD CENTRO POBLADO'] = cobertura_clean['COD CENTRO POBLADO'].fillna(0).astype(int)
cobertura_clean['COD MUNICIPIO'] = cobertura_clean['COD MUNICIPIO'].fillna(0).astype(int)

divipola_clean = df_datos_geograficos.copy()
divipola_clean['Longitud'] = divipola_clean['Longitud'].str.replace(',', '.').astype(float)
divipola_clean['Latitud'] = divipola_clean['Latitud'].str.replace(',', '.').astype(float)

df = cobertura_clean.merge(
    divipola_clean, 
    left_on='COD CENTRO POBLADO', 
    right_on='Código_Entidad', 
    how='left'
)
df.to_csv('data/cobertura_movil_con_datos_geograficos.csv', index=False)
df.head()

,AÑO,TRIMESTRE,PROVEEDOR,COD DEPARTAMENTO,DEPARTAMENTO,COD MUNICIPIO,MUNICIPIO,CABECERA MUNICIPAL,COD CENTRO POBLADO,CENTRO POBLADO,...,COBERTURA 5G,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
0,2023,3,COLOMBIA MOVIL S.A ESP,27,CHOCÓ,27250,EL LITORAL DEL SAN JUAN,N,27250034,TORDÓ,...,N,27.0,CHOCÓ,27250.0,EL LITORAL DEL SAN JUAN,27250034.0,TORDÓ,CP,-77.016686,4.337732
1,2023,3,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,5,ANTIOQUIA,5495,NECHÍ,N,5495003,LA CONCHA,...,N,5.0,ANTIOQUIA,5495.0,NECHÍ,5495003.0,LA CONCHA,CP,-74.868204,7.948383
2,2022,3,COLOMBIA MOVIL S.A ESP,70,SUCRE,70508,OVEJAS,N,70508006,DON GABRIEL,...,N,70.0,SUCRE,70508.0,OVEJAS,70508006.0,DON GABRIEL,CP,-75.293051,9.623491
3,2021,4,AVANTEL S.A.S,73,TOLIMA,73043,ANZOÁTEGUI,S,73043000,ANZOÁTEGUI,...,N,73.0,TOLIMA,73043.0,ANZOÁTEGUI,73043000.0,ANZOÁTEGUI,CM,-75.093772,4.631756
4,2021,2,COMUNICACION CELULAR S A COMCEL S A,50,META,50150,CASTILLA LA NUEVA,N,50150001,SAN LORENZO,...,N,50.0,META,50150.0,CASTILLA LA NUEVA,50150001.0,SAN LORENZO,CP,-73.548038,3.811495


In [5]:
def analizar_evolucion_cuota_mercado(df, tecnologia):
	resultado = []
	años = sorted(df['AÑO'].unique())
	
	for año in años:
		df_año = df[(df[tecnologia] == 'S') & (df['AÑO'] == año)]
		total = len(df_año)
		
		if total > 0:
			cuota = df_año.groupby('PROVEEDOR').size().reset_index(name='Centros')
			cuota['Porcentaje'] = (cuota['Centros'] / total) * 100
			cuota['AÑO'] = año
			resultado.append(cuota)
	
	if len(resultado) > 0:
		return pd.concat(resultado, ignore_index=True)
	else:
		return pd.DataFrame()

def analizar_cuota_mercado_por_tecnologia(df, tecnologia, año):
	df_tecnologia = df[(df[tecnologia] == 'S') & (df['AÑO'] == año)]
	cuota = df_tecnologia.groupby('PROVEEDOR').size().reset_index(name='Centros con cobertura')
	total = cuota['Centros con cobertura'].sum()
	if total > 0:
		cuota['Porcentaje'] = (cuota['Centros con cobertura'] / total) * 100
	else:
		cuota['Porcentaje'] = 0
	cuota = cuota.sort_values('Centros con cobertura', ascending=False)
	return cuota

def obtener_crecimiento_de_cobertura_por_ano(df, tecnologia):
	df_tecnologia = df[df[tecnologia] == 'S']
	df_tecnologia_por_ano = df_tecnologia.groupby('AÑO').size().reset_index(name='Cantidad de centros poblados con cobertura')
	return df_tecnologia_por_ano
	
tipos_de_tecnologia = [
    'COBERTURA 2G',
    'COBERTURA 3G',
    'COBERTURA HSPA+, HSPA+DC',
    'COBERTUTA 4G',
    'COBERTURA LTE',
    'COBERTURA 5G'
]
df_crecimiento_cobertura = {}

for tecnologia in tipos_de_tecnologia:
	df_crecimiento_cobertura[tecnologia] = obtener_crecimiento_de_cobertura_por_ano(df, tecnologia)
	
fig = go.Figure()

for tecnologia, df_tecnologia in df_crecimiento_cobertura.items():
	fig.add_trace(
		go.Scatter(
			x=df_tecnologia['AÑO'], 
			y=df_tecnologia['Cantidad de centros poblados con cobertura'], 
			mode='lines+markers', 
			name=tecnologia
		)
	)
fig.update_layout(
	title='Crecimiento de cobertura móvil por tecnología en Colombia',
	xaxis_title='Año',
	yaxis_title='Cantidad de centros poblados con cobertura',
	legend_title='Tecnología'
)
fig.write_html('graficos/crecimiento_cobertura_movil_por_tecnologia.html')
fig.show()



In [6]:
def obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(df, tecnologia, proveedor):
	df_tecnologia_proveedor = df[(df[tecnologia] == 'S') & (df['PROVEEDOR'] == proveedor)]
	df_tecnologia_proveedor_por_ano = df_tecnologia_proveedor.groupby('AÑO').size().reset_index(name='Cantidad de centros poblados con cobertura')
	return df_tecnologia_proveedor_por_ano


def mostrar_velocidad_adopcion_tecnologia_por_proveedor(df, tecnologia, proveedor):
	df_adopcion_tecnologia_2g = {}
	fig = go.Figure()
	for proveedor in proveedores:
		df_adopcion_tecnologia_2g[proveedor] = obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(df, 'COBERTURA 2G', proveedor)
	fig = go.Figure()
	for proveedor, df_proveedor in df_adopcion_tecnologia_2g.items():
		fig.add_trace(
			go.Scatter(
				x=df_proveedor['AÑO'], 
				y=df_proveedor['Cantidad de centros poblados con cobertura'], 
				mode='lines+markers', 
				name=proveedor
			)
		)
	fig.update_layout(
		title=f'Velocidad de adopción de {tecnologia} por {proveedor} en Colombia',
		xaxis_title='Año',
		yaxis_title='Cantidad de centros poblados con cobertura',
		legend_title='Proveedor'
	)
	tecnologia_clean = tecnologia.replace(' ', '_').lower()
	proveedor_clean = proveedor.replace(' ', '_').lower()
	fig.write_html(f'graficos/velocidad_adopcion_tecnologia_{tecnologia_clean}_por_{proveedor_clean}.html')
	fig.show()	


In [7]:
def mostrar_velocidad_adopcion_con_dropdown(df, tecnologias, proveedores):
    fig = go.Figure()
    
    for tecnologia in tecnologias:
        for proveedor in proveedores:
            df_data = obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(
                df, tecnologia, proveedor
            )
            fig.add_trace(
                go.Scatter(
                    x=df_data['AÑO'],
                    y=df_data['Cantidad de centros poblados con cobertura'],
                    mode='lines+markers',
                    name=proveedor,
                    visible=(tecnologia == tecnologias[0])  # Solo la primera visible
                )
            )
    
    buttons = []
    traces_per_tech = len(proveedores)
    
    for i, tecnologia in enumerate(tecnologias):
        visibility = [False] * len(fig.data)
        for j in range(i * traces_per_tech, (i + 1) * traces_per_tech):
            visibility[j] = True
        
        buttons.append(
            dict(
                label=tecnologia,
                method='update',
                args=[{'visible': visibility},
                      {'title': f'Adopción de {tecnologia} por proveedor'}]
            )
        )
    
    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons,
                direction='down',
                showactive=True,
                x=0.17,
                y=1.15
            )
        ],
        title=f'Adopción de {tecnologias[0]} por proveedor',
        xaxis_title='Año',
        yaxis_title='Centros poblados con cobertura'
    )
    
    fig.write_html('graficos/velocidad_adopcion_interactiva.html')
    fig.show()

proveedores = df['PROVEEDOR'].unique()
mostrar_velocidad_adopcion_con_dropdown(df, tipos_de_tecnologia, proveedores)
 

## Análisis de competencia entre proveedores

Cuota de mercado de cada proveedor por tecnología a lo largo del tiempo.

In [8]:
año_reciente = df['AÑO'].max()

todas_tecnologias = [col for col in df.columns if 'COBERTURA' in col or 'COBERTUTA' in col]


n_tecnologias = len(todas_tecnologias)
n_cols = 3 
n_rows = math.ceil(n_tecnologias / n_cols)

nombres_tecnologias = []
for tech in todas_tecnologias:
    nombre = tech.replace('COBERTURA ', '').replace('COBERTUTA ', '')
    nombres_tecnologias.append(nombre)

specs = [[{'type':'domain'} for _ in range(n_cols)] for _ in range(n_rows)]

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    specs=specs,
    subplot_titles=nombres_tecnologias
)

for idx, tecnologia in enumerate(todas_tecnologias):
    row = (idx // n_cols) + 1
    col = (idx % n_cols) + 1
    
    cuota = analizar_cuota_mercado_por_tecnologia(df, tecnologia, año_reciente)
    
    fig.add_trace(
        go.Pie(
            labels=cuota['PROVEEDOR'],
            values=cuota['Centros con cobertura'],
            name=nombres_tecnologias[idx],
            hovertemplate='%{label}<br>%{value} centros<br>%{percent}<extra></extra>'
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text=f'Cuota de Mercado por Proveedor - Todas las Tecnologías ({año_reciente})',
    height=400 * n_rows,
    showlegend=True
)

fig.write_html('graficos/cuota_mercado_todas_tecnologias.html')
fig.show()

In [9]:
todas_tecnologias = [col for col in df.columns if 'COBERTURA' in col or 'COBERTUTA' in col]

fig = go.Figure()

todos_proveedores = df['PROVEEDOR'].unique()

for idx, tecnologia in enumerate(todas_tecnologias):
    df_evolucion = analizar_evolucion_cuota_mercado(df, tecnologia)
    
    if len(df_evolucion) == 0:
        continue
    
    for proveedor in todos_proveedores:
        df_proveedor = df_evolucion[df_evolucion['PROVEEDOR'] == proveedor]
        
        if len(df_proveedor) > 0:
            fig.add_trace(
                go.Scatter(
                    x=df_proveedor['AÑO'],
                    y=df_proveedor['Porcentaje'],
                    name=proveedor,
                    mode='lines',
                    stackgroup='one', 
                    visible=(idx == 0),
                    hovertemplate='%{y:.1f}%<br>%{customdata} centros<extra></extra>',
                    customdata=df_proveedor['Centros']
                )
            )

buttons = []
traces_por_tecnologia = []

idx_actual = 0
for tecnologia in todas_tecnologias:
    df_evolucion = analizar_evolucion_cuota_mercado(df, tecnologia)
    if len(df_evolucion) == 0:
        traces_por_tecnologia.append((idx_actual, idx_actual))
        continue
    
    n_proveedores_con_datos = 0
    for proveedor in todos_proveedores:
        if len(df_evolucion[df_evolucion['PROVEEDOR'] == proveedor]) > 0:
            n_proveedores_con_datos += 1
    
    traces_por_tecnologia.append((idx_actual, idx_actual + n_proveedores_con_datos))
    idx_actual += n_proveedores_con_datos

for idx, tecnologia in enumerate(todas_tecnologias):
    nombre_limpio = tecnologia.replace('COBERTURA ', '').replace('COBERTUTA ', '')
    
    visibility = [False] * len(fig.data)
    inicio, fin = traces_por_tecnologia[idx]
    for i in range(inicio, fin):
        visibility[i] = True
    
    buttons.append(
        dict(
            label=nombre_limpio,
            method='update',
            args=[
                {'visible': visibility},
                {'title': f'Evolución de Cuota de Mercado en {nombre_limpio} por Proveedor'}
            ]
        )
    )

primera_tech = todas_tecnologias[0].replace('COBERTURA ', '').replace('COBERTUTA ', '')

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.17,
            y=1.15,
            xanchor='left',
            yanchor='top'
        )
    ],
    title=f'Evolución de Cuota de Mercado en {primera_tech} por Proveedor',
    xaxis_title='Año',
    yaxis_title='Cuota de Mercado (%)',
    hovermode='x unified',
    height=600
)

fig.write_html('graficos/evolucion_cuota_mercado_interactiva.html')
fig.show()

In [10]:
tecnologias_para_analizar = [col for col in df.columns if 'COBERTURA' in col or 'COBERTUTA' in col]

def identificar_lideres_mercado(df):
    años = sorted(df['AÑO'].unique())
    lideres = []
    
    for tecnologia in tecnologias_para_analizar:
        for año in años:
            try:
                cuota = analizar_cuota_mercado_por_tecnologia(df, tecnologia, año)
                if len(cuota) > 0:
                    lider = cuota.iloc[0]
                    lideres.append({
                        'AÑO': año,
                        'Tecnología': tecnologia.replace('COBERTURA ', '').replace('COBERTUTA ', ''),
                        'Líder': lider['PROVEEDOR'],
                        'Cuota (%)': lider['Porcentaje']
                    })
            except:
                pass
    
    return pd.DataFrame(lideres)

df_lideres = identificar_lideres_mercado(df)

for tecnologia in df_lideres['Tecnología'].unique():
    df_tech = df_lideres[df_lideres['Tecnología'] == tecnologia]

fig = px.bar(
    df_lideres,
    x='AÑO',
    y='Cuota (%)',
    color='Líder',
    facet_col='Tecnología',
    title='Cuota del Líder de Mercado por Tecnología',
    labels={'Cuota (%)': 'Cuota de Mercado del Líder (%)'}
)

fig.update_layout(height=400)
fig.write_html('graficos/lideres_mercado_por_tecnologia.html')
fig.show()

## Visualizaciones Geográficas de Cobertura Móvil

In [11]:
df_geo = pd.read_csv('data/cobertura_movil_con_datos_geograficos.csv', low_memory=False)
df_geo = df_geo[df_geo['Latitud'].notna() & df_geo['Longitud'].notna()]
año_reciente = df_geo['AÑO'].max()
print(f"Datos cargados: {len(df_geo)} registros con coordenadas válidas")
print(f"Año más reciente: {año_reciente}")

Datos cargados: 307399 registros con coordenadas válidas
Año más reciente: 2023


In [12]:
df_4g_reciente = df_geo[(df_geo['COBERTUTA 4G'] == 'S') & (df_geo['AÑO'] == año_reciente)]
muni_grp = df_4g_reciente.groupby(['MUNICIPIO', 'DEPARTAMENTO']).size().reset_index(name='Puntos_4G').sort_values(by='Puntos_4G', ascending=False)
muni_loc = pd.merge(muni_grp, df_4g_reciente[['MUNICIPIO', 'DEPARTAMENTO', 'Longitud', 'Latitud']].drop_duplicates(subset=['MUNICIPIO', 'DEPARTAMENTO']), on=['MUNICIPIO', 'DEPARTAMENTO'], how='left').drop_duplicates()

fig = px.scatter_map(
    muni_loc,
    lat='Latitud',
    lon='Longitud',
    size='Puntos_4G',
    color='Puntos_4G',
    hover_name='MUNICIPIO',
    hover_data={'DEPARTAMENTO': True, 'Puntos_4G': True, 'Latitud': ':.4f', 'Longitud': ':.4f'},
    text='MUNICIPIO',
    title=f'Cobertura 4G por Municipio ({año_reciente})',
    color_continuous_scale='Viridis',
    size_max=50,
    zoom=5
)
fig.update_traces(textposition='top center', textfont_size=8)
fig.update_layout(height=800, mapbox_style='open-street-map')
fig.write_html('graficos/mapa_cobertura_4g_municipios.html')
fig.show()

In [13]:
top_proveedores = df_4g_reciente.groupby('PROVEEDOR').size().nlargest(4).index
df_4g_top = df_4g_reciente[df_4g_reciente['PROVEEDOR'].isin(top_proveedores)]

prov_grp = df_4g_top.groupby(['PROVEEDOR', 'MUNICIPIO', 'DEPARTAMENTO']).size().reset_index(name='Puntos_Cobertura')
prov_loc = pd.merge(prov_grp, df_4g_top[['MUNICIPIO', 'DEPARTAMENTO', 'PROVEEDOR', 'Longitud', 'Latitud']].drop_duplicates(subset=['MUNICIPIO', 'DEPARTAMENTO', 'PROVEEDOR']), on=['MUNICIPIO', 'DEPARTAMENTO', 'PROVEEDOR'], how='left').drop_duplicates()

fig = px.scatter_map(
    prov_loc,
    lat='Latitud',
    lon='Longitud',
    size='Puntos_Cobertura',
    color='PROVEEDOR',
    hover_name='MUNICIPIO',
    hover_data={'DEPARTAMENTO': True, 'PROVEEDOR': True, 'Puntos_Cobertura': True, 'Latitud': ':.4f', 'Longitud': ':.4f'},
    title=f'Cobertura 4G por Proveedor - Top 4 Operadores ({año_reciente})',
    size_max=40,
    zoom=5
)
fig.update_layout(height=800, mapbox_style='open-street-map')
fig.write_html('graficos/mapa_proveedores_4g.html')
fig.show()

In [14]:
df_reciente = df_geo[df_geo['AÑO'] == año_reciente].copy()

def clasificar_tecnologia(row):
    if row['COBERTURA 5G'] == 'S':
        return '5G'
    elif row['COBERTURA LTE'] == 'S':
        return 'LTE'
    elif row['COBERTUTA 4G'] == 'S':
        return '4G'
    elif row['COBERTURA HSPA+, HSPA+DC'] == 'S':
        return 'HSPA+'
    elif row['COBERTURA 3G'] == 'S':
        return '3G'
    elif row['COBERTURA 2G'] == 'S':
        return '2G'
    else:
        return 'Sin cobertura'

df_reciente['Tecnologia'] = df_reciente.apply(clasificar_tecnologia, axis=1)

tech_grp = df_reciente.groupby(['MUNICIPIO', 'DEPARTAMENTO', 'Tecnologia']).size().reset_index(name='Centros')
tech_loc = pd.merge(tech_grp, df_reciente[['MUNICIPIO', 'DEPARTAMENTO', 'Tecnologia', 'Longitud', 'Latitud']].drop_duplicates(subset=['MUNICIPIO', 'DEPARTAMENTO', 'Tecnologia']), on=['MUNICIPIO', 'DEPARTAMENTO', 'Tecnologia'], how='left').drop_duplicates()

tech_order = ['Sin cobertura', '2G', '3G', 'HSPA+', '4G', 'LTE', '5G']
color_map = {
    'Sin cobertura': '#808080',
    '2G': '#ff0000',
    '3G': '#ff8800',
    'HSPA+': '#ffaa00',
    '4G': '#00cc00',
    'LTE': '#0088ff',
    '5G': '#8800ff'
}

fig = px.scatter_map(
    tech_loc,
    lat='Latitud',
    lon='Longitud',
    size='Centros',
    color='Tecnologia',
    hover_name='MUNICIPIO',
    hover_data={'DEPARTAMENTO': True, 'Tecnologia': True, 'Centros': True, 'Latitud': ':.4f', 'Longitud': ':.4f'},
    title=f'Distribución de Tecnologías por Municipio ({año_reciente})',
    category_orders={'Tecnologia': tech_order},
    color_discrete_map=color_map,
    size_max=35,
    zoom=5
)
fig.update_layout(height=800, mapbox_style='open-street-map')
fig.write_html('graficos/mapa_tecnologias.html')
fig.show()

In [15]:
df_sin_4g = df_reciente[df_reciente['COBERTUTA 4G'] == 'N'].copy()

sin4g_grp = df_sin_4g.groupby(['MUNICIPIO', 'DEPARTAMENTO']).size().reset_index(name='Centros_sin_4G')
sin4g_loc = pd.merge(sin4g_grp, df_sin_4g[['MUNICIPIO', 'DEPARTAMENTO', 'Longitud', 'Latitud']].drop_duplicates(subset=['MUNICIPIO', 'DEPARTAMENTO']), on=['MUNICIPIO', 'DEPARTAMENTO'], how='left').drop_duplicates()
sin4g_loc = sin4g_loc[sin4g_loc['Centros_sin_4G'] >= 3]

fig = px.scatter_map(
    sin4g_loc,
    lat='Latitud',
    lon='Longitud',
    size='Centros_sin_4G',
    color='Centros_sin_4G',
    hover_name='MUNICIPIO',
    hover_data={'DEPARTAMENTO': True, 'Centros_sin_4G': True, 'Latitud': ':.4f', 'Longitud': ':.4f'},
    text='MUNICIPIO',
    title=f'Brechas de Cobertura 4G - Municipios sin Cobertura ({año_reciente})',
    color_continuous_scale='Reds',
    size_max=50,
    zoom=5
)
fig.update_traces(textposition='top center', textfont_size=8)
fig.update_layout(height=800, mapbox_style='open-street-map')
fig.write_html('graficos/mapa_brechas_4g.html')
fig.show()

In [16]:
dept_grp = df_reciente.groupby('DEPARTAMENTO').size().reset_index(name='Total_Centros')
dept_loc = pd.merge(dept_grp, df_reciente[['DEPARTAMENTO', 'Longitud', 'Latitud']].drop_duplicates(subset='DEPARTAMENTO'), on='DEPARTAMENTO', how='left').drop_duplicates()

fig = px.scatter_map(
    dept_loc,
    lat='Latitud',
    lon='Longitud',
    size='Total_Centros',
    color='Total_Centros',
    hover_name='DEPARTAMENTO',
    hover_data={'Total_Centros': True, 'Latitud': ':.4f', 'Longitud': ':.4f'},
    text='DEPARTAMENTO',
    title=f'Centros Poblados por Departamento ({año_reciente})',
    color_continuous_scale='Plasma',
    size_max=60,
    zoom=5
)
fig.update_traces(textposition='top center', textfont_size=10)
fig.update_layout(height=800, mapbox_style='open-street-map')
fig.write_html('graficos/mapa_centros_departamento.html')
fig.show()